# Twitter Engagement Prediction

## Package Import

In [2]:
import sklearn
import emoji
import dateparser
import time
from scipy import sparse
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn import feature_extraction
from sklearn import decomposition
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import matplotlib as plt
import numpy as np
import string
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
import nltk
import pprint
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer
import random; random.seed(53)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import \
    StandardScaler, PolynomialFeatures,Normalizer
from sklearn.metrics import mean_squared_error

[nltk_data] Downloading package punkt to /Users/samsavage/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/samsavage/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Load Data  and Create Features

In [3]:
df = pd.read_csv("/Users/samsavage/Downloads/datasampleds_tosend_predict.csv")
#tokenize text
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
#tokenize,stem, and remove stop words from text
def cleanText(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text
#word count function
def word_count(string):
    split = string.split()
    count = len(split)
    return count
#count emojis
def count_emojis(string):
    em_count = len(re.findall(r'[^\w\s,.]', string))
    return em_count
#count hashtags
def count_hashtags(string):
    em_count = len(re.findall(r'[\#]', string))
    return em_count
#count mentions
def count_mentions(string):
    em_count = len(re.findall(r'[\@]', string))
    return em_count        
def text_analyzer(df):
    #feature creation{Word Count|character count|emoji count|link count}
    df['contents'] = df['contents'].astype('str')
    df['word_count'] = df['contents'].apply(word_count)
    df['character_count'] = [len(each) for each in df['contents']]
    df['link_count'] = df['contents'].str.count(r'(http)')
    df['emoji_count'] = df['contents'].apply(count_emojis)
    df['hashtag_count'] = df['contents'].apply(count_hashtags)
    df['mention_count'] = df['contents'].apply(count_mentions)
    #create other features including word count+sentiment based features
    sid = SentimentIntensityAnalyzer()
    df['polarity_pos'] = df['contents'].apply(lambda x: sid.polarity_scores(x)['pos'])
    df['polarity_neg'] = df['contents'].apply(lambda x: sid.polarity_scores(x)['neg'])
    df['polarity_neu'] = df['contents'].apply(lambda x: sid.polarity_scores(x)['neu'])
    df['polarity_compound'] = df['contents'].apply(lambda x: sid.polarity_scores(x)['compound'])
    return df
df = text_analyzer(df)
print(df.head(10))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samsavage/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   id                               partyId  networkAccountId  \
0   1  DF26B924-129B-4A94-A0AE-92153CB13A41        19212009.0   
1   2  4100D9EB-F2DA-418B-BA91-4CBDEA5A1AF9       108252113.0   
2   3  203B25B6-F5F4-4306-99AA-A22AE0BC0298          890891.0   
3   4  230A250C-173A-4EC6-A297-8CA06BDF213A       394216985.0   
4   5  ECB8582C-5E9E-4E13-A176-975AA4B98DC7        82652901.0   
5   6  0FDDC49F-640A-4A81-B8F6-2CDA629A631C        30278532.0   
6   7  613D6C16-1E0B-4954-BDBF-E8B20F711EDA        10005212.0   
7   8  391E0B22-79D9-45AD-B7D4-A84818479F25        14749070.0   
8   9  819E6F73-0CEB-4B1B-8B91-9F4006294C8D        36072985.0   
9  10  934F28DD-9F15-4A74-95B9-8A56A44B413B         5695632.0   

                                            contents postDate  engagements  \
0  On this week's @secondlifepod, Hillary chats w...  43:58.0          3.0   
1  Humble request to musicians, especially 'new f...  51:50.0        370.0   
2  .@dak and his brother embrace after he got pai.

## Extract relevent features and encode categorical varibles

In [4]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

df = df[['mediaType', 'effect1', 'effect2', 'effect3',
       'word_count', 'character_count', 'link_count', 'emoji_count','mention_count',
       'hashtag_count', 'polarity_pos', 'polarity_neg',
       'polarity_neu', 'polarity_compound','followers','engagements','contents']]
df = encode_and_bind(df,'mediaType')
df = encode_and_bind(df,'effect3')
print(df)

           effect1   effect2  word_count  character_count  link_count  \
0      81.77896455  1.309842          38              299           2   
1      118.1377431  0.799951          43              236           0   
2      174.1372666  1.359366          13              110           1   
3      64.05470599  0.900903           9               80           1   
4      78.58772967  1.447401          41              297           2   
...            ...       ...         ...              ...         ...   
99995  70.73702403  1.286623           3               31           1   
99996  272.2638861  0.912399          26              161           1   
99997  111.4691849  1.421489           9               73           0   
99998  112.9585951  0.791044          30              235           1   
99999  192.0863126  0.852310          19              136           1   

       emoji_count  mention_count  hashtag_count  polarity_pos  polarity_neg  \
0               15              2          

## Vectorize Text and combine with features

In [5]:
tfidf_vect = TfidfVectorizer(analyzer=cleanText)
tfidf_vect_fit = tfidf_vect.fit_transform(df['contents'].fillna(''))
tfidf_vect_fit_df = pd.DataFrame.sparse.from_spmatrix(tfidf_vect_fit)
tfidf_vect_fit_df
df = pd.concat([df,tfidf_vect_fit_df], axis=1)
print(df)

           effect1   effect2  word_count  character_count  link_count  \
0      81.77896455  1.309842          38              299           2   
1      118.1377431  0.799951          43              236           0   
2      174.1372666  1.359366          13              110           1   
3      64.05470599  0.900903           9               80           1   
4      78.58772967  1.447401          41              297           2   
...            ...       ...         ...              ...         ...   
99995  70.73702403  1.286623           3               31           1   
99996  272.2638861  0.912399          26              161           1   
99997  111.4691849  1.421489           9               73           0   
99998  112.9585951  0.791044          30              235           1   
99999  192.0863126  0.852310          19              136           1   

       emoji_count  mention_count  hashtag_count  polarity_pos  polarity_neg  \
0               15              2          

# Split Data into Validation,Test,Training datasets

In [6]:
validation = df.iloc[90001:,]
train = df.iloc[0:90000:,]
y = train['engagements'].fillna(0)
X = train.drop(columns={'engagements','contents'})
print(y.shape)
print(X.shape)


(90000,)
(90000, 222962)


### Split train

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


# ML Pipeline

## Baseline model

In [8]:
clf = LinearRegression()
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
print(predictions)
print(mean_squared_error(y_test,predictions,squared=False))

/Users/samsavage/mambaforge/envs/grailed_scraper/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/samsavage/mambaforge/envs/grailed_scraper/lib/python3.9/site-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


## Model Pipeline

In [ ]:
scoring = 'neg_mean_squared_error'
# Spot-Check Algorithms
models = []
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('Elastic', ElasticNet()))
models.append(('GBR', GradientBoostingRegressor()))
models.append(('SVR', SVR()))
models.append(('Decision', DecisionTreeRegressor()))
models.append(('MLP', MLPRegressor()))
seed = 5

##################################################
# evaluate each model in turn
results = []
names = []
score = 'neg_mean_squared_error'
# store preds
from sklearn.model_selection import cross_val_predict
dwPreds = []
for name, model in models:
 kfold = KFold(n_splits=10, random_state=seed, shuffle=True)
 # store the metrics
 cv_results = cross_val_score(model, X_train,y_train, cv=kfold, scoring = score)
 results.append(cv_results)
 names.append(name)
 msg = "%s: %f (%f)" % (name, abs(cv_results.mean()), cv_results.std())

### Algo Comparision Visulization

In [ ]:
# Compare Algorithms
fig = plt.figure(figsize=(22,10))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

NameError: name 'plt' is not defined

## Predicting Blank Engagment Records from Twitter Dataset